In [2]:
import block_methods as BM

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import integrate

plt.rcParams["figure.figsize"] = (16,9)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
n = 1000 # matrix dimension
b = 5 # block size
K = 10 # max iter

# set up matrix
lam = np.linspace(1e-2,1,n)
H = np.diag(lam)

# set up vector
V = np.random.randn(n,b)

In [7]:
# run Lanczos
Q,Qkp1,A,B,B_0 = BM.block_lanczos(H, V, K, K+1)

# generate tridiagonal matrix adn get eigendecomp
T = BM.get_block_tridiag(A,B)
Eval,Evec = np.linalg.eigh(T)

In [9]:
np.shape(B_0)[0]

5

In [7]:
# check orthogonality
np.max(np.abs(Q.T@Q-np.eye(b*K)))

1.2212453270876722e-15

In [8]:
# check three term recurrence
E = H@Q-Q@T
E[:,-b:] -= Qkp1@B[K-1]
np.max(np.linalg.norm(E,axis=0))

3.311051714667688e-16

In [38]:
z = -1
w = 0

Dzinv = np.diag(1/(Eval-z))
Dwinv = np.diag(1/(Eval-w))

# we can use (lam-z[:,None])*(block vector) instead of 
# np.diag(lam-z)@(block vector) to avoid forming an n \times n matrix
resz = V - (lam-z)[:,None]*(Q@Evec@Dzinv@Evec.T@BM.Ei(b*K,b,1)@B_0)
resw = V - (lam-w)[:,None]*(Q@Evec@Dwinv@Evec.T@BM.Ei(b*K,b,1)@B_0)

errz = 1/(lam-z)[:,None]*V - Q@Evec@Dzinv@Evec.T@BM.Ei(b*K,b,1)@B_0
errw = 1/(lam-w)[:,None]*V - Q@Evec@Dwinv@Evec.T@BM.Ei(b*K,b,1)@B_0

In [36]:
# check Lemma 1
Cz = BM.get_Cz(Eval,Evec,z,b,B_0)

np.linalg.norm(resz - Qkp1@B[K-1]@Cz),np.linalg.norm(resz)

(6.414187565707731e-14, 1.0016405161237455e-06)

In [39]:
# check Corollary 1
CwinvCz = BM.get_CwinvCz(Eval,Evec,z,w,b,B_0)

np.linalg.norm(resz-resw@CwinvCz)

6.414184770529834e-14

In [40]:
np.linalg.norm(errz-((lam-w)/(lam-z))[:,None]*(errw@CwinvCz))

4.09930456040736e-14